## Simulated_GRACE_universal
Averages model data onto GRACE mascon shapes - produces regridded bottom pressure files which will be uploaded

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import scipy.stats
import sklearn.metrics
import os
import time

In [2]:
import dask
from dask.distributed import Client
client = Client(n_workers=4)
client.amm.start()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 28,Total memory: 125.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33231,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 125.20 GiB
Comm: tcp://127.0.0.1:43835,Total threads: 7
Dashboard: /proxy/40535/status,Memory: 31.30 GiB
Nanny: tcp://127.0.0.1:44117,


In [4]:
# Read in data from 
import cosima_cookbook as cc
try:
    session = cc.database.create_session()
except:
    session = cc.database.create_session(db='/g/data/ik11/databases/cosima_master_2022-07-01.db')

In [5]:
#expt = '025deg_jra55_iaf_omip2_cycle5'
#expt = 'OM4_025.JRA_RYF'
#expt = '1deg_jra55_iaf_v2.0.0rc3'
expt = '01deg_jra55v140_iaf_cycle3'#'01deg_jra55v13_ryf9091'#
freq = '1 monthly'
grid_name = 'ACCESS-OM2-01'#'MOM6-025'#'ACCESS-OM2-1deg'#'ACCESS-OM2-025'#
mascon_name = 'JPL'#'ANU_300-3000split'#'GSFC'#'ANU_200'#'ANU_300'#'JPL'
time_slice = slice(None,None)

save_loc = '/g/data/x77/jj8842/'
dump_folder = 'regridded_pbot/jpl_regrid_accuracy/'
global_max_lat = 5

if (grid_name == 'ACCESS-OM2-01'):
    pbot_var = 'pbot_t'
    to_dbar = 1
    lon_var = 'xt_ocean'
    lat_var = 'yt_ocean'
    i_step = 450
    j_step = 225
    global_max_lon = 80
    global_min_lon = -280
elif (grid_name == 'ACCESS-OM2-025'):
    pbot_var = 'pbot_t'
    to_dbar = 1
    lon_var = 'xt_ocean'
    lat_var = 'yt_ocean'
    i_step = 225
    j_step = 225
    global_max_lon = 80
    global_min_lon = -280
elif (grid_name == 'ACCESS-OM2-1deg'):
    pbot_var = 'pbot_t'
    to_dbar = 1
    lon_var = 'xt_ocean'
    lat_var = 'yt_ocean'
    i_step = 90
    j_step = 140
    global_max_lon = 80
    global_min_lon = -280
elif grid_name == 'MOM6-025':
    pbot_var = 'pbo'
    to_dbar = 1/10**4
    lon_var = 'xh'
    lat_var = 'yh'
    i_step = 90
    j_step = 140
    assert False, "Please put global max lon in here"
else:
    assert False, 'Can\'t find some default variables'

In [6]:
model_pbot = cc.querying.getvar(expt,pbot_var, session,frequency = freq)*to_dbar
model_pbot = model_pbot.sel(time=time_slice)

In [7]:
grid_to_mascon = xr.load_dataset(save_loc+'/mascon_definitions/'+grid_name+'_grid__'+mascon_name+'.nc').primary_mascon.astype(int)

In [8]:
mascon_centres = xr.load_dataset(save_loc+'mascon_definitions/centres_'+mascon_name+'.nc')
lons = mascon_centres.lons
lats = mascon_centres.lats
mascons = mascon_centres.mc

In [9]:
# Get the grids to line up
lons[lons>global_max_lon] = lons[lons>global_max_lon]-360
lons[lons<global_min_lon] = lons[lons<global_min_lon]+360

In [10]:
output_list=[]
no_lons = len(model_pbot[lon_var])
no_lats = np.max(np.where(model_pbot[lat_var]<global_max_lat)[0])
for i in range(0,no_lons,i_step):
    for j in range(0,no_lats,j_step):
        outfilepath = save_loc+dump_folder+expt+'__'+mascon_name+str(i)+'_'+str(j)+'.nc'
        if os.path.isfile(outfilepath):
            outdata = xr.load_dataset(outfilepath)
        else:
            ### What mascons do I care about?
            lon_min = model_pbot[lon_var].isel({lon_var:i})
            lat_min = model_pbot[lat_var].isel({lat_var:j})

            if i+i_step >= no_lons:
                lon_max = global_max_lon
            else:
                lon_max = model_pbot[lon_var].isel({lon_var:i+i_step})

            if j+j_step >= no_lats:
                lat_max = global_max_lat
            else:
                lat_max = model_pbot[lat_var].isel({lat_var:j+j_step})



            bi = (lats>lat_min)&(lats<=lat_max) & (lons>lon_min) & (lons<lon_max)
            useful_mascons = mascons.where(bi,drop=True)
            
            if len(useful_mascons) == 0:
                continue

            simulated_mascons = model_pbot.broadcast_like(useful_mascons).where(grid_to_mascon == useful_mascons,drop=True).mean((lon_var,lat_var))
            simulated_mascons.load()

            outdata = xr.Dataset({'ewh':simulated_mascons,
                              'lon':lons.where(bi,drop=True),
                              'lat':lats.where(bi,drop=True)},
                            attrs = {'original_model':expt, 'units':'dbar'})
            outdata.to_netcdf(outfilepath)
            
        output_list.append(outdata)
    #client.restart()

In [12]:
full_output = xr.concat(output_list,'mc','all')
full_output.to_netcdf(save_loc+'regridded_pbot/'+expt+'__'+mascon_name+'.nc')